# this is a _test_

In [1]:
import pandas as pd

In [17]:
print(3)

3


In [ ]:
print(pd)

In [ ]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);

In [ ]:
print(data)

In [ ]:
data_text = data[['headline_text']]
print(data_text)

In [6]:
data_text['index'] = data_text.index


In [7]:
print(data_text)

                                             headline_text    index
0        aba decides against community broadcasting lic...        0
1           act fire witnesses must be aware of defamation        1
2           a g calls for infrastructure protection summit        2
3                 air nz staff in aust strike for pay rise        3
4            air nz strike to affect australian travellers        4
5                        ambitious olsson wins triple jump        5
6               antic delighted with record breaking barca        6
7        aussie qualifier stosur wastes four memphis match        7
8             aust addresses un security council over iraq        8
9               australia is locked into war timetable opp        9
10       australia to contribute 10 million in aid to iraq       10
11       barca take record as robson celebrates birthda...       11
12                              bathhouse plans move ahead       12
13           big hopes for launceston cycling ch

In [8]:
documents = data_text

In [9]:
print(len(documents))

1103663


In [10]:
print(documents[:5])

                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [11]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexanderbollbach/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [13]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [14]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [ ]:
print(3)

In [32]:
processed_docs = documents['headline_text'].map(preprocess)


dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
print(dictionary)

print("foo")

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit
Dictionary(62374 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)
foo


In [33]:
print(dictionary)

Dictionary(62374 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)


In [35]:
print(len(dictionary))
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
print(len(dictionary))

62374
14174


In [36]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4021, 1)]

In [37]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4021 ("dampen") appears 1 time.


In [38]:
print(bow_doc_4310)

[(76, 1), (112, 1), (483, 1), (4021, 1)]


In [41]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903603121911333),
 (1, 0.3852450692300274),
 (2, 0.4974556050119205),
 (3, 0.505567858418396)]


In [42]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

print(lda_model)

LdaModel(num_terms=14174, num_topics=10, decay=0.5, chunksize=2000)


In [43]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"queensland" + 0.020*"countri" + 0.018*"hospit" + 0.017*"tasmanian" + 0.017*"hour" + 0.015*"work" + 0.014*"children" + 0.013*"student" + 0.013*"citi" + 0.011*"prison"
Topic: 1 
Words: 0.030*"australia" + 0.023*"world" + 0.017*"market" + 0.014*"sydney" + 0.012*"record" + 0.012*"share" + 0.011*"victoria" + 0.010*"australian" + 0.009*"port" + 0.008*"year"
Topic: 2 
Words: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.014*"adelaid" + 0.014*"west" + 0.014*"attack" + 0.013*"price" + 0.013*"gold"
Topic: 3 
Words: 0.027*"govern" + 0.017*"plan" + 0.014*"council" + 0.013*"rural" + 0.013*"say" + 0.012*"indigen" + 0.012*"turnbul" + 0.011*"water" + 0.011*"chang" + 0.011*"commun"
Topic: 4 
Words: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.014*"leagu" + 0.014*"lose" + 0.011*"john" + 0.010*"life" + 0.010*"make" + 0.009*"premier" + 0.009*"unit"
Topic: 5 
Words: 0.027*"elect" + 0.023*"south" + 0.017*"final" + 0.016*"open" + 0.014*"austra

In [44]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.008*"grandstand" + 0.007*"octob" + 0.005*"festiv" + 0.005*"univers" + 0.005*"music" + 0.004*"year" + 0.004*"footag" + 0.004*"rememb" + 0.004*"islam" + 0.004*"anzac"
Topic: 1 Word: 0.007*"friday" + 0.006*"thursday" + 0.006*"june" + 0.005*"say" + 0.005*"updat" + 0.005*"know" + 0.005*"human" + 0.004*"grain" + 0.004*"scott" + 0.004*"data"
Topic: 2 Word: 0.009*"leagu" + 0.009*"final" + 0.009*"australia" + 0.009*"donald" + 0.009*"world" + 0.006*"beat" + 0.006*"juli" + 0.006*"rugbi" + 0.005*"win" + 0.005*"open"
Topic: 3 Word: 0.015*"charg" + 0.014*"murder" + 0.014*"interview" + 0.012*"court" + 0.010*"podcast" + 0.009*"jail" + 0.009*"child" + 0.009*"alleg" + 0.008*"sentenc" + 0.008*"accus"
Topic: 4 Word: 0.022*"rural" + 0.015*"news" + 0.012*"drum" + 0.008*"nation" + 0.007*"centr" + 0.006*"andrew" + 0.005*"dairi" + 0.005*"detent" + 0.004*"shop" + 0.004*"homeless"
Topic: 5 Word: 0.008*"sport" + 0.008*"christma" + 0.007*"health" + 0.007*"michael" + 0.006*"mental" + 0.006*"ash" + 

In [45]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5856690406799316	 
Topic: 0.008*"weather" + 0.007*"farm" + 0.007*"marriag" + 0.006*"energi" + 0.005*"hunter" + 0.005*"coal" + 0.005*"queensland" + 0.005*"john" + 0.005*"peter" + 0.004*"korea"

Score: 0.25430071353912354	 
Topic: 0.009*"leagu" + 0.009*"final" + 0.009*"australia" + 0.009*"donald" + 0.009*"world" + 0.006*"beat" + 0.006*"juli" + 0.006*"rugbi" + 0.005*"win" + 0.005*"open"

Score: 0.020005833357572556	 
Topic: 0.012*"market" + 0.008*"turnbul" + 0.007*"share" + 0.006*"rise" + 0.006*"price" + 0.005*"coast" + 0.005*"water" + 0.005*"septemb" + 0.005*"australian" + 0.005*"dollar"

Score: 0.020004671066999435	 
Topic: 0.015*"crash" + 0.015*"polic" + 0.012*"kill" + 0.009*"woman" + 0.009*"die" + 0.009*"dead" + 0.009*"miss" + 0.008*"shoot" + 0.008*"search" + 0.008*"death"

Score: 0.020004665479063988	 
Topic: 0.022*"rural" + 0.015*"news" + 0.012*"drum" + 0.008*"nation" + 0.007*"centr" + 0.006*"andrew" + 0.005*"dairi" + 0.005*"detent" + 0.004*"shop" + 0.004*"homeless"

Score

In [46]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34999993443489075	 Topic: 0.032*"charg" + 0.030*"court" + 0.023*"murder" + 0.022*"polic" + 0.019*"face"
Score: 0.3499816358089447	 Topic: 0.027*"elect" + 0.023*"south" + 0.017*"final" + 0.016*"open" + 0.014*"australia"
Score: 0.18333332240581512	 Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.013*"jail"
Score: 0.016679609194397926	 Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china"
Score: 0.01667213998734951	 Topic: 0.027*"govern" + 0.017*"plan" + 0.014*"council" + 0.013*"rural" + 0.013*"say"
Score: 0.01666666753590107	 Topic: 0.032*"queensland" + 0.020*"countri" + 0.018*"hospit" + 0.017*"tasmanian" + 0.017*"hour"
Score: 0.01666666753590107	 Topic: 0.030*"australia" + 0.023*"world" + 0.017*"market" + 0.014*"sydney" + 0.012*"record"
Score: 0.01666666753590107	 Topic: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.014*"leagu" + 0.014*"lose"
Score: 0.01666666753590107	 Topic: 0.023*"hous" + 0.019*"die" + 0.015*"h